## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import pandas as pd
from datetime import datetime
import time 
import requests

## <span style='color:#ff5f27'> 👮🏻‍♂️ API Key

In [2]:
API_KEY = 'YOUR_API_KEY'

date_today = datetime.now().strftime("%Y-%m-%d")

## <span style='color:#ff5f27'> 🧑🏻‍🏫 Functions

In [3]:
def get_json(city, date, API_KEY=API_KEY):
    return requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city.lower()}/{date}?unitGroup=metric&include=days&key={API_KEY}&contentType=json')\
            .json()


def get_data(city_name,date=date_today):
    json = get_json(city_name,date)
    data = json['days'][0]
    
    return [
        json['address'].capitalize(),
        [json['latitude'],json['longitude']],
        data['datetime'],
        data['tempmax'],
        data['tempmin'],
        data['temp'],
        data['feelslikemax'],
        data['feelslikemin'],
        data['feelslike'],
        data['dew'],
        data['humidity'],
        data['precip'],
        data['precipprob'],
        data['precipcover'],
        data['snow'],
        data['snowdepth'],
        data['windgust'],
        data['windspeed'],
        data['winddir'],
        data['pressure'],
        data['cloudcover'],
        data['visibility'],
        data['solarradiation'],
        data['solarenergy'],
        data['uvindex'],
        data['conditions']
    ]


def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

## <span style='color:#ff5f27'> 🕵🏻‍♂️ Parsing

In [4]:
cities = ['Kyiv','Stockholm','Sundsvall','Malmo']

data_parsed = [get_data(city) for city in cities]

## <span style='color:#ff5f27'> 🧑🏻‍🏫 Dataset Preparation

In [5]:
col_names = [
    'city',
    'location',
    'date',
    'tempmax',
    'tempmin',
    'temp',
    'feelslikemax',
    'feelslikemin',
    'feelslike',
    'dew',
    'humidity',
    'precip',
    'precipprob',
    'precipcover',
    'snow',
    'snowdepth',
    'windgust',
    'windspeed',  
    'winddir',
    'pressure',
    'cloudcover', 
    'visibility',
    'solarradiation',
    'solarenergy',
    'uvindex',
    'conditions'
]

In [6]:
new_data = pd.DataFrame(
    data_parsed,
    columns=col_names
)
new_data.date = new_data.date.apply(timestamp_2_time)

new_data.head()

,city,location,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Kyiv,"[50.4506, 30.5243]",1662498000000,17.7,4.6,11.5,17.7,4.6,11.5,1.8,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
1,Stockholm,"[59.3326, 18.0668]",1662498000000,15.9,7.8,12.0,15.9,7.1,11.8,7.0,...,25.2,13.0,70.7,1022.0,59.5,15.3,132.5,11.6,5.0,Partially cloudy
2,Sundsvall,"[62.3889, 17.3067]",1662498000000,13.0,3.0,8.6,13.0,0.1,7.4,5.5,...,31.0,14.3,192.9,1024.1,90.8,15.3,116.1,10.1,5.0,Overcast
3,Malmo,"[55.5967, 13.0011]",1662498000000,20.6,12.4,15.9,20.6,12.4,15.9,9.3,...,44.6,23.7,97.9,1018.4,69.0,15.2,157.3,13.7,5.0,Partially cloudy


## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [7]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/167
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 👩🏻‍🔬 Retrieving or Creating Feature Group</span>

In [8]:
def get_or_create_feature_group(name='weather_fg',fs=fs):
    feature_group = fs.get_or_create_feature_group(
        name=name,
        description = 'Characteristics of each day',
        version = 1,
        primary_key = ['index'],
        online_enabled = True,
        event_time = ['date']
    )
    return feature_group

In [9]:
feature_group = get_or_create_feature_group()

feature_group.insert(new_data.reset_index())

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/167/fs/109/fg/786


Uploading Dataframe: 0.00% |          | Rows 0/4 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/167/jobs/named/weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fc40e7cb2e0>, None)

---